# PHD - DISC24 - Exp 2 - KL for random circuits - native gates - v1 - 20240406

//======================================================================================================
// 

EPUSP - Quantum circuits dimensions - v.04.2024 
Author: Waldemir Cambiucci 
Data: April 4th, 2024

File: PHD - DISC24 - Exp 2 - KL for random circuits - native gates - v1 - 20240406

Description: 

https://www.calculatorsoup.com/calculators/discretemathematics/combinations.php


//
//======================================================================================================

In [28]:
import re
import networkx as nx
from qiskit import QuantumCircuit
from networkx.algorithms import community

In [29]:
def extract_qubits(input_str):
    # Regular expression to match qubit numbers
    pattern = r'q\[(\d+)\]'
    # Find all matches of the pattern in the input string
    matches = re.findall(pattern, input_str)
    # Extract qubit numbers from matches
    qubit_numbers = [int(match) for match in matches]
    # Return the qubit numbers as separate variables
    if len(qubit_numbers) >= 2:
        return tuple(qubit_numbers)
    elif len(qubit_numbers) == 1:
        return qubit_numbers[0], None
    else:
        return None, None

In [30]:
def get_num_qubits_from_qasm(qasm_str):  
    for line in qasm_str.splitlines():
        if line.startswith('qreg'):        
            pos = line.find('[') + 1
            end_pos = line.find(']')
            qubits_str = line[pos:end_pos]    
    return int(qubits_str)

In [145]:
import networkx as nx
def convert_qasm_in_hypergraph_kl(qasm_str):
    g = nx.MultiGraph()     
    for line in qasm_str.splitlines():      
        if line.startswith('cx'):        
            qubit1, qubit2 = extract_qubits(line)      
            g.add_edge(qubit1,qubit2)      
    return g           

In [146]:
def count_ebits(g , b):      
    nebits = 0
    for edge in g.edges:
        n1 = edge[0]
        n2 = edge[1]        
        if (n1 in b[0]) and (n2 in b[1]):   
            nebits = nebits + 1
        elif (n1 in b[1]) and (n2 in b[0]):   
            nebits = nebits + 1
    return nebits

In [147]:
def get_coupling_base_from_circuit(circuit):
    n = circuit.num_qubits
    Cb = (n * (n-1))/2  
    return Cb

In [34]:
def get_coupling_ratio_from_circuit(circuit):
    num_bin_gates = count_binary_operations(circuit)
    Cb = get_coupling_base_from_circuit(circuit)
    Cr = num_bin_gates / Cb
    return Cr

In [35]:
def count_binary_operations(quantum_circuit):
    # Count the operations using count_ops() function
    ops_dict = quantum_circuit.count_ops()    
    # Sum the counts of binary operations
    total_binary_ops = sum(count for op, count in ops_dict.items() if op in ['cx', 'cz', 'swap', 'cp'])    
    return total_binary_ops

In [36]:
def create_full_circuit(n):
    bc = QuantumCircuit(n)    
    total_unique_gates = (n*(n-1))/2     
    for q0 in range(0,n):
        bc.barrier()
        for q1 in range(q0+1,n):
            bc.cz(q0,q1)    
    return bc

In [37]:
def split_circuit_and_count_binary_gates(circuit):
    # Get the number of qubits in the circuit
    num_qubits = circuit.num_qubits
    
    # Calculate the midpoint to split the circuit
    midpoint = num_qubits // 2
    
    # Divide the qubits into two partitions
    partition_1 = list(range(midpoint))    
    partition_2 = list(range(midpoint, num_qubits))
    
    # Initialize count for binary gates involving qubits from both partitions
    count_binary_gates = 0
    
    # Iterate through the gates in the circuit
    for instr, qargs, cargs in circuit.data:
        # Check if the gate is a binary gate and involves qubits from both partitions
        if len(qargs) == 2 and (qargs[0].index in partition_1 and qargs[1].index in partition_2) or \
           (qargs[0].index in partition_2 and qargs[1].index in partition_1):
            count_binary_gates += 1
    
    return count_binary_gates

In [73]:
from qiskit import QuantumCircuit
def split_circuit_and_count_binary_gates(circuit):
    # Get the number of qubits in the circuit
    num_qubits = circuit.num_qubits
    
    # Calculate the midpoint to split the circuit
    midpoint = num_qubits // 2
    
    # Divide the qubits into two partitions
    partition_1 = set(range(midpoint))    
    partition_2 = set(range(midpoint, num_qubits))
    
    # Initialize count for binary gates involving qubits from both partitions
    count_binary_gates = 0
    
    # Iterate through each gate in the circuit
    for gate in circuit:
        # Extract the qubits associated with the gate
        qubits_involved = {qubit.index for qubit in gate[1]}
        
        # Check if the gate is a binary gate and involves qubits from both partitions
        if len(qubits_involved) == 2 and (qubits_involved & partition_1) and (qubits_involved & partition_2):
            count_binary_gates += 1
    
    return count_binary_gates

# Testing functions with big circuits

In [56]:
test_circuit = create_full_circuit(4)

In [57]:
n = test_circuit.num_qubits
print(n)

4


In [58]:
test_circuit.draw()

░           ░        ░     ░ 
q_0: ─░──■──■──■──░────────░─────░─
      ░  │  │  │  ░        ░     ░ 
q_1: ─░──■──┼──┼──░──■──■──░─────░─
      ░     │  │  ░  │  │  ░     ░ 
q_2: ─░─────■──┼──░──■──┼──░──■──░─
      ░        │  ░     │  ░  │  ░ 
q_3: ─░────────■──░─────■──░──■──░─
      ░           ░        ░     ░

In [80]:
qasm_str = test_circuit.qasm()
qasm_hypergraph = convert_qasm_in_hypergraph_kl(qasm_str)

In [81]:
qasm_hypergraph.nodes

NodeView((0, 1, 2, 3))

In [82]:
print(qasm_hypergraph.edges)

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]


In [62]:
blocks = community.kernighan_lin.kernighan_lin_bisection(qasm_hypergraph, partition=None, max_iter=50)
print(blocks)

({0, 1}, {2, 3})


In [63]:
num_ebits = count_ebits(qasm_hypergraph,blocks)
print('total ebits with KL bipartition: ',num_ebits)

total ebits with KL bipartition:  4


In [64]:
split_circuit_and_count_binary_gates(test_circuit)

C:\Users\wcamb\AppData\Local\Temp\ipykernel_26448\1118521752.py:19: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  qubits_involved = {qubit.index for qubit in gate[1]}


4

# outro teste...

In [139]:
# Example usage:
circuit_str = """
// Benchmark was created by MQT Bench on 2024-03-18
// For more information about MQT Bench, please visit https://www.cda.cit.tum.de/mqtbench/
// MQT Bench version: 1.1.0
// Qiskit version: 1.0.2
// Used Gate Set: ['id', 'rz', 'sx', 'x', 'cx', 'measure', 'barrier']

OPENQASM 2.0;
include "qelib1.inc";
qreg q[4];
creg meas[4];
cx q[2],q[1];
cx q[2],q[1];
cx q[2],q[1];
cx q[3],q[1];
cx q[2],q[1];
cx q[1],q[3];
cx q[0],q[3];
cx q[1],q[3];
cx q[0],q[3];
cx q[0],q[1];
cx q[0],q[1];
cx q[3],q[2];
cx q[3],q[2];
cx q[0],q[3];
cx q[0],q[3];
cx q[2],q[0];
cx q[2],q[0];
cx q[1],q[0];
cx q[1],q[0];
cx q[2],q[3];
cx q[2],q[3];
cx q[2],q[3];
cx q[1],q[2];
cx q[2],q[1];
cx q[3],q[0];
cx q[3],q[0];
barrier q[0],q[1],q[2],q[3];
measure q[0] -> meas[0];
measure q[1] -> meas[1];
measure q[2] -> meas[2];
measure q[3] -> meas[3];
"""

In [148]:
test_qc = QuantumCircuit.from_qasm_str(circuit_str)
print(test_qc)

                                                                         ┌───┐»
   q_0: ────────────────────────────────■─────────■────■────■────■────■──┤ X ├»
        ┌───┐┌───┐┌───┐┌───┐┌───┐       │         │  ┌─┴─┐┌─┴─┐  │    │  └─┬─┘»
   q_1: ┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├──■────┼────■────┼──┤ X ├┤ X ├──┼────┼────┼──»
        └─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘  │    │    │    │  ├───┤├───┤  │    │    │  »
   q_2: ──■────■────■────┼────■────┼────┼────┼────┼──┤ X ├┤ X ├──┼────┼────■──»
                         │       ┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐└─┬─┘└─┬─┘┌─┴─┐┌─┴─┐     »
   q_3: ─────────────────■───────┤ X ├┤ X ├┤ X ├┤ X ├──■────■──┤ X ├┤ X ├─────»
                                 └───┘└───┘└───┘└───┘          └───┘└───┘     »
meas: 4/══════════════════════════════════════════════════════════════════════»
                                                                              »
«        ┌───┐┌───┐┌───┐          ┌───┐     ┌───┐ ░ ┌─┐         
«   q_0: ┤ X ├┤ X ├┤ X ├──────────┤ X ├─────┤ X ├─░─┤M├

In [149]:
split_circuit_and_count_binary_gates(test_qc)

C:\Users\wcamb\AppData\Local\Temp\ipykernel_26448\1118521752.py:19: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  qubits_involved = {qubit.index for qubit in gate[1]}


17

In [150]:
print(test_qc.num_qubits)

4


In [151]:
qasm_str = test_qc.qasm()
qasm_hypergraph = convert_qasm_in_hypergraph_kl(qasm_str)

In [152]:
print(qasm_hypergraph.edges)

[(2, 1, 0), (2, 1, 1), (2, 1, 2), (2, 1, 3), (2, 1, 4), (2, 1, 5), (2, 3, 0), (2, 3, 1), (2, 3, 2), (2, 3, 3), (2, 3, 4), (2, 0, 0), (2, 0, 1), (1, 3, 0), (1, 3, 1), (1, 3, 2), (1, 0, 0), (1, 0, 1), (1, 0, 2), (1, 0, 3), (3, 0, 0), (3, 0, 1), (3, 0, 2), (3, 0, 3), (3, 0, 4), (3, 0, 5)]


In [153]:
print(qasm_hypergraph.nodes)

[2, 1, 3, 0]


In [154]:
blocks = community.kernighan_lin.kernighan_lin_bisection(qasm_hypergraph, partition=None, max_iter=50)
print(blocks)

({0, 3}, {1, 2})


In [155]:
num_ebits = count_ebits(qasm_hypergraph,blocks)
print('total ebits with KL bipartition: ',num_ebits)

total ebits with KL bipartition:  14


# DISC24 - Exp 2 - KL for random circuits with native gates

In [156]:
from datetime import datetime
def generate_timestamp():
    """
    Generates a timestamp string representing the current date and time.    
    Returns:
        str: Timestamp string in the format 'YYYY-MM-DD_HH-MM-SS'.
    """
    now = datetime.now()
    timestamp = now.strftime('%Y-%m-%d_%H-%M-%S')
    return timestamp

In [157]:
def disc_exp2_random_circuits_native_gates(opt):
    
    timestamp = generate_timestamp()
    print("Timestamp:", timestamp)

    # Folder of circuit files for benchmark.
    circuit_folder_opt = 'disc_exp2_random_native_circuits\\opt'+str(opt)

    print('Running experiment 2 - random circuits with native gates - calculating w, s, Cb, min.cut, and segments using KL partitioning.')
    print('Optimization - opt: ' + str(opt))
    print('----------------------------------------------------------------------------------------------------------')
    print('...')

    file1 = open(circuit_folder_opt + "\\" + 'exp_2_random_native_opt' + str(opt) + '-' + timestamp + '.log', 'w')
    
    # Write the contents of the list to a file named 'coupling.log'
    file1.writelines('random native circuit' + '\t' + 'width (n)' + '\t' + 'depth (d)' + '\t' + 'size (s)' + '\t' + 'coupling (Cb)' + '\t' + 'mid.cut' + '\t' + 'min.cut KL'+ '\t' + 'bipartite segments' + '\n')


    #=====================================================================
    #
    for circuit_file_index in range(4,121):

        random_file_name = 'random_nativegates_ibm_qiskit_opt' + str(opt) + '_' + str(circuit_file_index) + '.qasm'        
        print('Random native circuit :', random_file_name)

        # loading qasm instructions for the circuit.
        circuit = QuantumCircuit.from_qasm_file( circuit_folder_opt + "\\" + random_file_name )            

        d = circuit.depth()
        s = circuit.size()       
        n = circuit.num_qubits
        
        mid_cuts = split_circuit_and_count_binary_gates(circuit)
        
        Cb = get_coupling_base_from_circuit(circuit)    
        Rc = get_coupling_ratio_from_circuit(circuit)        
        
        qasm_str = circuit.qasm()
        qasm_hypergraph = convert_qasm_in_hypergraph_kl(qasm_str)

        blocks = community.kernighan_lin.kernighan_lin_bisection(qasm_hypergraph, partition=None, max_iter=50)
        num_ebits = count_ebits(qasm_hypergraph,blocks)
        
        print(qasm_hypergraph)

        file1.writelines( random_file_name + '\t' + str(n) + '\t' + str(d) + '\t' + str(s) + '\t' + str(Cb) + '\t' + str(mid_cuts) + '\t' + str(num_ebits) + '\t' + str(blocks) + '\n' )

    #
    #=====================================================================

    print('...')
    print('File closed.')

    file1.close()
    

In [158]:
disc_exp2_random_circuits_native_gates(0)

Timestamp: 2024-04-06_13-18-36
Running experiment 2 - random circuits with native gates - calculating w, s, Cb, min.cut, and segments using KL partitioning.
Optimization - opt: 0
----------------------------------------------------------------------------------------------------------
...
Random native circuit : random_nativegates_ibm_qiskit_opt0_4.qasm
MultiGraph with 4 nodes and 26 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_5.qasm
MultiGraph with 5 nodes and 40 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_6.qasm
MultiGraph with 6 nodes and 49 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_7.qasm
MultiGraph with 7 nodes and 69 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_8.qasm
MultiGraph with 8 nodes and 93 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_9.qasm
MultiGraph with 9 nodes and 110 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_10.qasm
MultiGraph with 10 nodes

C:\Users\wcamb\AppData\Local\Temp\ipykernel_26448\1118521752.py:19: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  qubits_involved = {qubit.index for qubit in gate[1]}


MultiGraph with 11 nodes and 225 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_12.qasm
MultiGraph with 12 nodes and 285 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_13.qasm
MultiGraph with 13 nodes and 311 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_14.qasm
MultiGraph with 14 nodes and 366 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_15.qasm
MultiGraph with 15 nodes and 466 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_16.qasm
MultiGraph with 16 nodes and 568 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_17.qasm
MultiGraph with 17 nodes and 496 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_18.qasm
MultiGraph with 18 nodes and 611 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_19.qasm
MultiGraph with 19 nodes and 682 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_20.qasm
MultiGraph with 20 nodes and 783 edges
Random 

MultiGraph with 88 nodes and 16059 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_89.qasm
MultiGraph with 89 nodes and 16464 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_90.qasm
MultiGraph with 90 nodes and 17064 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_91.qasm
MultiGraph with 91 nodes and 17582 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_92.qasm
MultiGraph with 92 nodes and 17668 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_93.qasm
MultiGraph with 93 nodes and 18259 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_94.qasm
MultiGraph with 94 nodes and 18622 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_95.qasm
MultiGraph with 95 nodes and 18788 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_96.qasm
MultiGraph with 96 nodes and 19335 edges
Random native circuit : random_nativegates_ibm_qiskit_opt0_97.qasm
MultiGraph with 97 nodes and

In [159]:
disc_exp2_random_circuits_native_gates(1)

Timestamp: 2024-04-06_13-24-10
Running experiment 2 - random circuits with native gates - calculating w, s, Cb, min.cut, and segments using KL partitioning.
Optimization - opt: 1
----------------------------------------------------------------------------------------------------------
...
Random native circuit : random_nativegates_ibm_qiskit_opt1_4.qasm
MultiGraph with 4 nodes and 26 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_5.qasm
MultiGraph with 5 nodes and 38 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_6.qasm
MultiGraph with 6 nodes and 49 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_7.qasm
MultiGraph with 7 nodes and 69 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_8.qasm
MultiGraph with 8 nodes and 93 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_9.qasm
MultiGraph with 9 nodes and 110 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_10.qasm
MultiGraph with 10 nodes

C:\Users\wcamb\AppData\Local\Temp\ipykernel_26448\1118521752.py:19: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  qubits_involved = {qubit.index for qubit in gate[1]}


MultiGraph with 11 nodes and 223 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_12.qasm
MultiGraph with 12 nodes and 285 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_13.qasm
MultiGraph with 13 nodes and 309 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_14.qasm
MultiGraph with 14 nodes and 366 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_15.qasm
MultiGraph with 15 nodes and 466 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_16.qasm
MultiGraph with 16 nodes and 568 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_17.qasm
MultiGraph with 17 nodes and 496 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_18.qasm
MultiGraph with 18 nodes and 611 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_19.qasm
MultiGraph with 19 nodes and 682 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_20.qasm
MultiGraph with 20 nodes and 783 edges
Random 

MultiGraph with 88 nodes and 16057 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_89.qasm
MultiGraph with 89 nodes and 16458 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_90.qasm
MultiGraph with 90 nodes and 17062 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_91.qasm
MultiGraph with 91 nodes and 17578 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_92.qasm
MultiGraph with 92 nodes and 17666 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_93.qasm
MultiGraph with 93 nodes and 18257 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_94.qasm
MultiGraph with 94 nodes and 18620 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_95.qasm
MultiGraph with 95 nodes and 18780 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_96.qasm
MultiGraph with 96 nodes and 19331 edges
Random native circuit : random_nativegates_ibm_qiskit_opt1_97.qasm
MultiGraph with 97 nodes and

In [160]:
disc_exp2_random_circuits_native_gates(2)

Timestamp: 2024-04-06_13-30-47
Running experiment 2 - random circuits with native gates - calculating w, s, Cb, min.cut, and segments using KL partitioning.
Optimization - opt: 2
----------------------------------------------------------------------------------------------------------
...
Random native circuit : random_nativegates_ibm_qiskit_opt2_4.qasm
MultiGraph with 4 nodes and 26 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_5.qasm
MultiGraph with 5 nodes and 36 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_6.qasm
MultiGraph with 6 nodes and 49 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_7.qasm
MultiGraph with 7 nodes and 69 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_8.qasm
MultiGraph with 8 nodes and 93 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_9.qasm
MultiGraph with 9 nodes and 110 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_10.qasm


C:\Users\wcamb\AppData\Local\Temp\ipykernel_26448\1118521752.py:19: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  qubits_involved = {qubit.index for qubit in gate[1]}


MultiGraph with 10 nodes and 140 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_11.qasm
MultiGraph with 11 nodes and 223 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_12.qasm
MultiGraph with 12 nodes and 285 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_13.qasm
MultiGraph with 13 nodes and 309 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_14.qasm
MultiGraph with 14 nodes and 366 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_15.qasm
MultiGraph with 15 nodes and 466 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_16.qasm
MultiGraph with 16 nodes and 568 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_17.qasm
MultiGraph with 17 nodes and 496 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_18.qasm
MultiGraph with 18 nodes and 611 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_19.qasm
MultiGraph with 19 nodes and 682 edges
Random 

MultiGraph with 87 nodes and 15965 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_88.qasm
MultiGraph with 88 nodes and 16055 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_89.qasm
MultiGraph with 89 nodes and 16454 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_90.qasm
MultiGraph with 90 nodes and 17058 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_91.qasm
MultiGraph with 91 nodes and 17576 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_92.qasm
MultiGraph with 92 nodes and 17666 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_93.qasm
MultiGraph with 93 nodes and 18251 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_94.qasm
MultiGraph with 94 nodes and 18620 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_95.qasm
MultiGraph with 95 nodes and 18776 edges
Random native circuit : random_nativegates_ibm_qiskit_opt2_96.qasm
MultiGraph with 96 nodes and

In [161]:
disc_exp2_random_circuits_native_gates(3)

Timestamp: 2024-04-06_13-47-35
Running experiment 2 - random circuits with native gates - calculating w, s, Cb, min.cut, and segments using KL partitioning.
Optimization - opt: 3
----------------------------------------------------------------------------------------------------------
...
Random native circuit : random_nativegates_ibm_qiskit_opt3_4.qasm
MultiGraph with 4 nodes and 25 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_5.qasm
MultiGraph with 5 nodes and 26 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_6.qasm
MultiGraph with 6 nodes and 46 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_7.qasm
MultiGraph with 7 nodes and 63 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_8.qasm
MultiGraph with 8 nodes and 83 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_9.qasm
MultiGraph with 9 nodes and 100 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_10.qasm


C:\Users\wcamb\AppData\Local\Temp\ipykernel_26448\1118521752.py:19: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  qubits_involved = {qubit.index for qubit in gate[1]}


MultiGraph with 10 nodes and 125 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_11.qasm
MultiGraph with 11 nodes and 196 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_12.qasm
MultiGraph with 12 nodes and 273 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_13.qasm
MultiGraph with 13 nodes and 296 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_14.qasm
MultiGraph with 14 nodes and 342 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_15.qasm
MultiGraph with 15 nodes and 445 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_16.qasm
MultiGraph with 16 nodes and 544 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_17.qasm
MultiGraph with 17 nodes and 480 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_18.qasm
MultiGraph with 18 nodes and 592 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_19.qasm
MultiGraph with 19 nodes and 638 edges
Random 

MultiGraph with 87 nodes and 15566 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_88.qasm
MultiGraph with 88 nodes and 15628 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_89.qasm
MultiGraph with 89 nodes and 16040 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_90.qasm
MultiGraph with 90 nodes and 16634 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_91.qasm
MultiGraph with 91 nodes and 17106 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_92.qasm
MultiGraph with 92 nodes and 17242 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_93.qasm
MultiGraph with 93 nodes and 17794 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_94.qasm
MultiGraph with 94 nodes and 18145 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_95.qasm
MultiGraph with 95 nodes and 18323 edges
Random native circuit : random_nativegates_ibm_qiskit_opt3_96.qasm
MultiGraph with 96 nodes and